In [1]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input, UpSampling2D
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = '../input/stage1_train/'
TEST_PATH = '../input/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using TensorFlow backend.


In [2]:
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [ ]:
# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask
# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
def get_UNet(filters = 64, act = 'relu', pretrained_weights = None, prob = 0.1):
    
    inputs = Input(shape = (IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))
    pad = 
    # contraction: encoder
    conv1 = Conv2D(filters, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(inputs)
    conv1 = Dropout(prob) (conv1)
    conv1 = Conv2D(filters, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(filters * 2, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(pool1)
    conv2 = Dropout(prob) (conv2)    
    conv2 = Conv2D(filters * 2, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(filters * 4, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(pool2)
    conv3 = Dropout(prob) (conv3)
    conv3 = Conv2D(filters * 4, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(filters * 8, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(pool3)
    conv4 = Dropout(prob) (conv4)
    conv4 = Conv2D(filters * 8, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)   
    
    conv5 = Conv2D(filters * 16, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(pool4)
    conv5 = Dropout(prob) (conv5)
    conv5 = Conv2D(filters * 16, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(conv5)
    
    # expansion: decoder
    up6 = Conv2DTranspose(filters * 8, 2, strides = 2, padding='same')(conv5)
    up6 = concatenate([conv4, up6])
    conv6 = Conv2D(filters * 8, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(up6)
    conv6 = Dropout(prob) (conv6)    
    conv6 = Conv2D(filters * 8, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(conv6)
    
    up7 = Conv2DTranspose(filters * 4, 2, strides = 2, padding='same')(conv6)
    up7 = concatenate([conv3, up7])
    conv7 = Conv2D(filters * 4, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(up7)
    conv7 = Dropout(prob) (conv7)
    conv7 = Conv2D(filters * 4, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(conv7)
        
    up8 = Conv2DTranspose(filters * 2, 2, strides = 2, padding='same')(conv7)
    up8 = concatenate([conv2, up8])
    conv8 = Conv2D(filters * 2, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(up8)
    conv8 = Dropout(prob) (conv8)
    conv8 = Conv2D(filters * 2, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(conv8)
        
    up9 = Conv2DTranspose(filters, 2, strides = 2, padding='same')(conv8)
    up9 = concatenate([conv1, up9])
    conv9 = Conv2D(filters, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(up9)
    conv9 = Dropout(prob) (conv9)
    conv9 = Conv2D(filters, 3, activation = act, padding = pad, kernel_initializer = 'he_normal')(conv9)
    
    outputs = Conv2D(1, 1, activation = 'sigmoid', padding = pad, kernel_initializer = 'he_normal')(conv9)

    model = Model(inputs = [inputs], outputs = [outputs])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics = [mean_iou])
    model.summary()
    
    if(pretrained_weights):
        model.load_weights(pretrained_weights)
    return model

